In [2]:
import pickle
import gradio as gr
from data_preprocessor  import DataPreprocessor
import pandas as pd

with open('../src/preprocessor.pkl', 'rb') as f:
    preprocessor = pickle.load(f)

with open('../src/model.pkl', 'rb') as f:
    model = pickle.load(f)

schema = preprocessor.schema

d:\code stuffs\Python\vietai-project-tmp\car-price-prediction-dvm\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
cat_choices = preprocessor.get_categorical_choices()

inputs = []
input_names = []

for feat in schema.values:
    featname = feat[0]
    if feat[2] == 'none':
        inputs.append(gr.Number(label=featname))
    elif featname == 'genmodel':
        inputs.append(gr.Dropdown([item for sublist in cat_choices['genmodel'].values() for item in sublist], label = featname))
    else:
        choices = cat_choices.get(featname, ['Other'])
        inputs.append(gr.Dropdown(choices, label=featname))

    input_names.append(featname)

def predict_gradio(*args):
    X = pd.DataFrame([args], columns=input_names)
    X = preprocessor.perform_light_cleaning(X)
    X = preprocessor.impute(X)
    X = preprocessor.encode(X)  
    X = X[preprocessor.get_encoded_cols()]  
    X = preprocessor.scale(X)
    y_pred = model.predict(X)[0]
    
    return float(y_pred)

demo = gr.Interface(
    fn=predict_gradio,
    inputs=inputs,
    outputs=gr.Number(label="Predicted Price"),
    title="Car Price Prediction"
)

demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
